In [1]:
import numpy as np
import pandas as pd
import pulp
from ortoolpy import addvars, addbinvars
import random

In [2]:
# ペナルティの重み定数
p_need_diff = 1000
p_amount = 10
p_veteran = 3

In [3]:
# ここでExcelのデータを取得
shift = pd.read_csv('prototype/Shift.tsv', sep='\t',  index_col=0)
manage = pd.read_csv('prototype/Manage.tsv', sep='\t', index_col=0)
member = pd.read_csv('prototype/Member.tsv', sep='\t', index_col=0).T
# detail = pd.read_csv('prototype/Detail.tsv', sep='\t', index_col=0)

In [4]:
# 入れる場所が1、入れない場所が0を表す
shift.head()

,Aさん,Bさん,Cさん,Dさん
1,1,1,1,0
2,1,1,1,0
3,1,1,1,1


In [5]:
manage.head()

,need
1,2
2,2
3,3


In [6]:
member.head()

,amount,veteran
Aさん,2,1
Bさん,2,1
Cさん,2,1
Dさん,1,1


In [7]:
# Shiftシートのカラムとインデックス
date = shift.index
employee = shift.columns
print('シフト日時')
print(date)
print('従業員')
print(employee)

シフト日時
Int64Index([1, 2, 3], dtype='int64')
従業員
Index(['Aさん', 'Bさん', 'Cさん', 'Dさん'], dtype='object')


In [8]:
shift['need_employee'] = manage.need  # １シフトあたりの必要人数の列を追加
shift.head()

,Aさん,Bさん,Cさん,Dさん,need_employee
1,1,1,1,0,2
2,1,1,1,0,2
3,1,1,1,1,3


In [9]:
# 足りない人がいれば終了
shortage = []
for day, row in shift.iterrows():
    print('{}日\tシフト入れる人数: {}, 必要人数: {}'.format(day, row[employee].sum(), row['need_employee']))
    if row[employee].sum() < row['need_employee']:
        shortage.append(day)

if shortage:
    print('下記日付について人員が足りません')
    print('日,'.join(shortage))

1日	シフト入れる人数: 3, 必要人数: 2
2日	シフト入れる人数: 3, 必要人数: 2
3日	シフト入れる人数: 4, 必要人数: 3


In [10]:
# 従業員数と月の日数
num_date, num_employee = shift[employee].shape
print('シフト日数: {}, 従業員数: {}'.format(num_date, num_employee))

シフト日数: 3, 従業員数: 4


In [11]:
# 最適化のための変数作成
var = pd.DataFrame(np.array(addbinvars(num_date, num_employee)), columns=employee, index=date)
var.head()

,Aさん,Bさん,Cさん,Dさん
1,v000001,v000002,v000003,v000004
2,v000005,v000006,v000007,v000008
3,v000009,v000010,v000011,v000012


In [12]:
# ０と１を逆転させている
shift_rev = shift[employee].apply(lambda x: 1 - x, axis=1)
# 入れる場所が0、入れない場所が1を表す
shift_rev.head()

,Aさん,Bさん,Cさん,Dさん
1,0,0,0,1
2,0,0,0,1
3,0,0,0,0


In [13]:
k = pulp.LpProblem('shift', sense=pulp.LpMinimize)

In [14]:
# 制約：希望していない場所には入らないようにする
for (_, s), (_, v) in zip(shift_rev.iterrows(), var.iterrows()):
    # シフト表とシフト変数の内積(要素積)を計算する
    # 入れない場所に1が立っているため、変数が1をとると入れない場所に入れることになってしまうため0となるように制約を指定している
    k += pulp.lpDot(s, v) <= 0

In [15]:
# 変数の追加
var['need_employee'] = addvars(num_date)  # 必要人数を満たすためのペナルティ変数
var['num_veteran'] = addvars(num_date)  # 新人だけになった時のペナルティ変数
var.head()

,Aさん,Bさん,Cさん,Dさん,need_employee,num_veteran
1,v000001,v000002,v000003,v000004,v000013,v000016
2,v000005,v000006,v000007,v000008,v000014,v000017
3,v000009,v000010,v000011,v000012,v000015,v000018


In [16]:
# 制約: シフトに必要な人数に対するペナルティーを求める。
for (_, s), (_, v) in zip(shift.iterrows(), var.iterrows()):
    k += v['need_employee'] >= (pulp.lpSum(v[employee]) - s['need_employee'])
    k += v['need_employee'] >= -(pulp.lpSum(v[employee]) - s['need_employee'])

In [17]:
# 入りすぎ、入らなすぎにペナルティを与える変数
shift_amount = pd.DataFrame(addvars(num_employee), index=employee).T
shift_amount

,Aさん,Bさん,Cさん,Dさん
0,v000019,v000020,v000021,v000022


In [18]:
shift_amount.iloc[0]

Aさん    v000019
Bさん    v000020
Cさん    v000021
Dさん    v000022
Name: 0, dtype: object

In [19]:
# 制約
for name, r in var[employee].iteritems():
    k += pulp.lpSum(r) + shift_amount.at[0, name] >= member.at[name, 'amount']
    k += pulp.lpSum(r) - shift_amount.at[0, name] <= member.at[name, 'amount']

In [20]:
# 目的関数決定
k += p_need_diff * pulp.lpSum(var['need_employee']) \
     + p_amount * pulp.lpSum(shift_amount.iloc[0])

In [21]:
print(k)

shift:
MINIMIZE
1000*v000013 + 1000*v000014 + 1000*v000015 + 10*v000019 + 10*v000020 + 10*v000021 + 10*v000022 + 0
SUBJECT TO
_C1: v000004 <= 0

_C2: v000008 <= 0

_C3:0 <= 0

_C4: - v000001 - v000002 - v000003 - v000004 + v000013 >= -2

_C5: v000001 + v000002 + v000003 + v000004 + v000013 >= 2

_C6: - v000005 - v000006 - v000007 - v000008 + v000014 >= -2

_C7: v000005 + v000006 + v000007 + v000008 + v000014 >= 2

_C8: - v000009 - v000010 - v000011 - v000012 + v000015 >= -3

_C9: v000009 + v000010 + v000011 + v000012 + v000015 >= 3

_C10: v000001 + v000005 + v000009 + v000019 >= 2

_C11: v000001 + v000005 + v000009 - v000019 <= 2

_C12: v000002 + v000006 + v000010 + v000020 >= 2

_C13: v000002 + v000006 + v000010 - v000020 <= 2

_C14: v000003 + v000007 + v000011 + v000021 >= 2

_C15: v000003 + v000007 + v000011 - v000021 <= 2

_C16: v000004 + v000008 + v000012 + v000022 >= 1

_C17: v000004 + v000008 + v000012 - v000022 <= 1

VARIABLES
0 <= v000001 <= 1 Integer
0 <= v000002 <= 1 Integer

In [22]:
k.solve()

1

In [23]:
result = np.vectorize(pulp.value)(var[employee]).astype(int)
print('目的関数', pulp.value(k.objective))
print(result)

目的関数 0.0
[[0 1 1 0]
 [1 0 1 0]
 [1 1 0 1]]


In [40]:
df_result = pd.DataFrame(result, columns=employee, index=date)
df_result.loc['Total'] = df_result.sum(numeric_only=True)
df_result['Total'] = df_result.sum(numeric_only=True, axis=1)
df_result = pd.merge(df_result, manage, left_index=True, right_index=True, how='outer')
df_result = df_result.append(member['amount'])
df_result

,Aさん,Bさん,Cさん,Dさん,Total,need
1,0.0,1.0,1.0,0.0,2.0,2.0
2,1.0,0.0,1.0,0.0,2.0,2.0
3,1.0,1.0,0.0,1.0,3.0,3.0
Total,2.0,2.0,2.0,1.0,7.0,NaN
amount,2.0,2.0,2.0,1.0,NaN,NaN
